In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('Input.xlsx')

In [ ]:
urls = []
url_id = []
num_of_urls = len(df['URL'])
for i in range(num_of_urls):
  urls.append(df['URL'][i])
  url_id.append(df['URL_ID'][i])

In [ ]:
# Import Required Module
import requests
from bs4 import BeautifulSoup

for i in range (num_of_urls):
  #print(url_id[i])
  # Web URL
  Web_url = urls[i]

  # Get URL Content
  r = requests.get(Web_url)

  # Parse HTML Code
  soup = BeautifulSoup(r.content, 'html.parser')

  headings = soup.find_all(['h1','tdb-title-text'])

  tags = soup.find_all('div', class_=["td-post-content tagdiv-type","td_block_wrap tdb_single_content tdi_130 td-pb-border-top td_block_template_1 td-post-content tagdiv-type"])

  file_path = str(url_id[i])+'.txt'
  file = open(file_path, 'w')
  for heading in headings:
    file.write(heading.text)  # Writing in file
  for tag in tags:
    file.write(tag.text)
  file.close()  # Close the file


In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
stopWord1 = open('StopWords_Auditor.txt','r')
stopWord2 = open('StopWords_Currencies.txt','r',encoding="latin-1")
stopWord3 = open('StopWords_DatesandNumbers.txt','r')
stopWord4 = open('StopWords_Generic.txt','r')
stopWord5 = open('StopWords_GenericLong.txt','r')
stopWord6 = open('StopWords_Geographic.txt','r')
stopWord7 = open('StopWords_Names.txt','r')

In [ ]:
s1 = stopWord1.readlines()
s2 = stopWord2.readlines()
s3 = stopWord3.readlines()
s4 = stopWord4.readlines()
s5 = stopWord5.readlines()
s6 = stopWord6.readlines()
s7 = stopWord7.readlines()

In [ ]:
import re

In [ ]:
stopWords = []
for s in [s1,s2,s3,s4,s5,s6,s7]:
  string = []
  for ch in s:
    x =re.split('[;|,|"|?|:| |.|\||\n|$|%|!|£|#|@|]',ch[:-1])
    stopWords.append(x)

In [ ]:
stopWord = []
for i in stopWords:
  for j in i:
    if len(j)!=0:
      stopWord.append(j.upper())

In [ ]:
stopWord1.close()
stopWord2.close()
stopWord3.close()
stopWord4.close()
stopWord5.close()
stopWord6.close()
stopWord7.close()

In [ ]:
all_files = []
for i in range(num_of_urls):
  file = open(str(url_id[i])+'.txt','r')
  f = file.read()
  a = nltk.word_tokenize(f)
  all_files.append(a)
  file.close()


In [ ]:
s = 0
for i in all_files:
  s += len(i)
print(s)


145501


In [ ]:
def removingStopWords(fileList):
  newList = []
  for word in fileList:
    if word.upper() not in stopWord:
      if word not in ['.','!',',','%','-','(','’',':','|']:
        newList.append(word)
  return newList



In [ ]:
updatedList = []
for file in all_files:
  updatedList.append(removingStopWords(file))


In [ ]:
s = 0
for i in updatedList:
  s += len(i)
print(s)

58692


In [ ]:
updatedList = []
for file in all_files:
  updatedList.append(removingStopWords(file))


In [ ]:
def countingPositiveWords(fileList):
  positiveCount = 0
  for word in fileList:
    if word.lower() in pWordsList:
      #print('true')
      positiveCount = positiveCount+1
  return positiveCount

In [ ]:
count = 0
pScore = []
for file in updatedList:
  x =countingPositiveWords(file)
  pScore.append(x)
  #print(url_id[count], x)
  count = count + 1

In [ ]:
nFile = open('negative-words.txt','r',encoding = 'ISO-8859-1')
nWords = nFile.readlines()
nWordsList = []
for n in nWords:
  if len(n[:-1]) != 0:
    nWordsList.append(n[:-1].lower())

nFile.close()

In [ ]:
def countingNegativeWords(fileList):
  negativeCount = 0
  for word in fileList:
    if word.lower() in nWordsList:
      #print('true')
      negativeCount = negativeCount-1
  return negativeCount*-1

In [ ]:
count = 0
nScore = []
for file in updatedList:
  x =countingNegativeWords(file)
  nScore.append(x)
  #print(url_id[count], x)
  count = count + 1

In [ ]:
def polarityScore(index):
  return (pScore[index] - nScore[index]) / ((pScore[index] + nScore[index]) + 0.000001)


In [ ]:
polScore = []
for i in range(num_of_urls):
  polScore.append(polarityScore(i))


In [ ]:
def subjectiveScore(index):
  return(pScore[index] + nScore[index])/(len(updatedList[index])+0.000001)

In [ ]:
subScore = []
for i in range(num_of_urls):
  subScore.append(subjectiveScore(i))

In [ ]:
from openpyxl import load_workbook


In [ ]:
workbook = load_workbook(filename='Output Data Structure.xlsx')

In [ ]:
worksheet = workbook['Sheet1']

In [ ]:
def find_cell_coordinates(worksheet, cell_value):
    for row in worksheet.iter_rows():
        for cell in row:
            if cell.value == cell_value:
                return cell.row, cell.column


In [ ]:
cell_value = 'POSITIVE SCORE'
row_num, column_num = find_cell_coordinates(worksheet, cell_value)
for r in range(1,num_of_urls+1):
  worksheet.cell(row=row_num+r, column=column_num).value = pScore[r-1]


In [ ]:
cell_value = 'NEGATIVE SCORE'
row_num, column_num = find_cell_coordinates(worksheet, cell_value)
for r in range(1,num_of_urls+1):
  worksheet.cell(row=row_num+r, column=column_num).value = nScore[r-1]

In [ ]:
cell_value = 'POLARITY SCORE'
row_num, column_num = find_cell_coordinates(worksheet, cell_value)
for r in range(1,num_of_urls+1):
  worksheet.cell(row=row_num+r, column=column_num).value = polScore[r-1]

In [ ]:
cell_value = 'SUBJECTIVITY SCORE'
row_num, column_num = find_cell_coordinates(worksheet, cell_value)
for r in range(1,num_of_urls+1):
  worksheet.cell(row=row_num+r, column=column_num).value = subScore[r-1]

2

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
allFiles = []
for i in range(num_of_urls):
  file = open(str(url_id[i])+'.txt','r')
  f = file.read()
  stop_words = set(stopwords.words('english'))
  stop_words.add(',')
  stop_words.add('!')
  stop_words.add('?')
  stop_words.add('-')
  stop_words.add('(')
  stop_words.add(')')
  stop_words.add(':')
  stop_words.add(';')
  stop_words.add('“')
  stop_words.add('”')
  stop_words.add("'")
  word_tokens = word_tokenize(f)
  a = [w for w in word_tokens if not w.lower() in stop_words]
  allFiles.append(a)
  file.close()


In [ ]:
def ppCount(c):
  personPronoun = 0
  file = open(str(url_id[c])+'.txt','r')
  f = file.read()
  x = re.split('[ |\n]',f)
  for i in x:
    if i.upper() in ['I','WE','US''MY','OURS']:
          personPronoun += 1
  return personPronoun

In [ ]:
def countingFunction(lst,c):
  words = 0
  sentences = 0
  complexWords = 0
  syllablePerWord = 0
  personPronoun = 0
  wordLength = 0
  for i in lst:
    if i in ['.',' . ',' .','. ']:

      sentences += 1
    elif i not in ['.',' . ',' .','. ']:
      words += 1
      syllable_count = 0
      vowels = 'aeiouy'
      wordLength += len(i)
      if i[0] in vowels:
          syllable_count += 1
      l = len(i)
      for index in range(1, l):
          if i[index] in vowels and i[index - 1] not in vowels:
              syllable_count += 1
      if i.endswith('e'):
          syllable_count -= 1
      if i.endswith('le') and l > 2 and i[-3] not in vowels:
          syllable_count += 1
      if syllable_count == 0:
          syllable_count += 1
      syllablePerWord += syllable_count
      if syllable_count > 2:
        complexWords += 1
      personPronoun = ppCount(c)

  return words, sentences, complexWords, syllablePerWord, personPronoun, wordLength


In [ ]:
avgSenLenLst = []
perComplexWordsLst = []
fogIndexLst = []
avgNoWordPerSenLst = []
complexWordCountLst = []
wordCountLst = []
syllablePerWordLst = []
personalPronounLst = []
avgWordLenLst = []
length = len(allFiles)
for i in range(length):
  words, sentences, complexWords, syllablePerWord, personPronoun, wordLength = countingFunction(allFiles[i],i)
  if sentences == 0 or words == 0:
    avgSenLenLst.append(0)
    perComplexWordsLst.append(0)
    fogIndexLst.append(0)
    avgNoWordPerSenLst.append(0)
    complexWordCountLst.append(0)
    wordCountLst.append(0)
    syllablePerWordLst.append(0)
    personalPronounLst.append(0)
    avgWordLenLst.append(0)

  else:
    avgSenLenLst.append(words/sentences)
    perComplexWordsLst.append(complexWords/words)
    fogIndexLst.append(0.4*(avgSenLenLst[i] + perComplexWordsLst[i]))
    avgNoWordPerSenLst.append(words/sentences)
    complexWordCountLst.append(complexWords)
    wordCountLst.append(words)
    syllablePerWordLst.append(syllablePerWord)
    personalPronounLst.append(personPronoun)
    avgWordLenLst.append(wordLength/words)



In [ ]:
cell_value = 'AVG SENTENCE LENGTH'
row_num, column_num = find_cell_coordinates(worksheet, cell_value)
for r in range(1,num_of_urls+1):
  worksheet.cell(row=row_num+r, column=column_num).value = avgSenLenLst[r-1]
  worksheet.cell(row=row_num+r, column=column_num+1).value = perComplexWordsLst[r-1]
  worksheet.cell(row=row_num+r, column=column_num+2).value = fogIndexLst[r-1]
  worksheet.cell(row=row_num+r, column=column_num+3).value = avgNoWordPerSenLst[r-1]
  worksheet.cell(row=row_num+r, column=column_num+4).value = complexWordCountLst[r-1]
  worksheet.cell(row=row_num+r, column=column_num+5).value = wordCountLst[r-1]
  worksheet.cell(row=row_num+r, column=column_num+6).value = syllablePerWordLst[r-1]
  worksheet.cell(row=row_num+r, column=column_num+7).value = personalPronounLst[r-1]
  worksheet.cell(row=row_num+r, column=column_num+8).value = avgWordLenLst[r-1]


In [ ]:
workbook.save(filename='Output Data Structure.xlsx')